<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [10]</a>'.</span>

In [1]:
from IPython.display import Code

# Example: Comparison of RISC-V SW Toolchains with MLonMCU

TODO

## Supported components

**Models:** Any (`toycar` used below)

**Frontends:** Any (`tflite` used below)

**Frameworks/Backends:** Any (`tvmaotplus` used below)

**Platforms/Targets:** `etiss`, `spike`, `ovpsim`,... (`etiss` used below)

## Prerequisites

If not done already, setup a virtual python environment and install the required packages into it. (See `requirements.txt`)

In [2]:
Code(filename="requirements.txt")

-r ../../requirements.txt

Set up MLonmCU as usual, i.e. initialize an environment and install all required dependencies. Feel free to use the following minimal `environment.yml.j2` template:

In [3]:
Code(filename="environment.yml.j2")

---
# The MLONMCU_HOME is filled in automatically when creating the environment
home: "{{ home_dir }}"
logging:
  level: INFO
  to_file: false
  rotate: false
cleanup:
  auto: true
  keep: 5
# Default locations for certain directoriescan be changed here
# Non-absolute paths will always be threated relative to the MLONMCU_HOME
paths:
  # Where the dependencies are downloaded and installed
  deps: deps
  # If logging to file is used keep logs in this directory
  logs: logs
  # Location where reports and artifacts are written to
  results: results
  # Directory where custom extensions can be integrated
  plugins: plugins
  # Directory for intermediate build products, should be located on a large enough drive
  temp: temp
  # A collection of models which will be used to look for models
  # The paths will be checked in the order defined here stopping at the first match
  # Non-existant paths will be skipped without throwing an error
  models:
    - "{{ home_dir }}/models"
# Here default clone_urls
repos:
  tvm:
    url: "https://github.com/apache/tvm.git"
    ref: v0.17.0
    options:
      recursive: true
  mlif:
    url: "https://github.com/tum-ei-eda/mlonmcu-sw.git"
    ref: c7b0e7364c232666ebd5a17476181e3ff9a71d70
  spike:
    url: "https://github.com/riscv-software-src/riscv-isa-sim.git"
    ref: 0bc176b3fca43560b9e8586cdbc41cfde073e17a
  spikepk:
    url: "https://github.com/riscv-software-src/riscv-pk.git"
    ref: 7e9b671c0415dfd7b562ac934feb9380075d4aa2
frameworks:
  default: tvm
  tvm:
    enabled: true
    backends:
      default: tvmaotplus
      tvmaotplus:
        enabled: true
        features: {}
    features: {}
frontends:
  tflite:
    enabled: true
    features: []
toolchains:
  gcc: true
  llvm: true
platforms:
  mlif:
    enabled: true
    features:
      debug: true
      vext: true
      auto_vectorize: true
targets:
  default: host_x86
  spike:
    enabled: true
    features:
      vext: true
  host_x86:
    enabled: true
    features: []
postprocesses:
  use: []
vars:
  allow_extensions: false
  runs_per_stage: true
  llvm.dl_url: "https://syncandshare.lrz.de/dl/fiWBtDLWz17RBc1Yd4VDW7/LLVM/default/18.1.8/Ubuntu/{{ ubuntu_version if ubuntu_version is defined else '20.04'}}/llvm.tar.xz"
  riscv_gcc.dl_url: "https://syncandshare.lrz.de/dl/fiWBtDLWz17RBc1Yd4VDW7/GCC/default/2024.09.03_gcc14/Ubuntu/{{ ubuntu_version if ubuntu_version is defined else '20.04'}}/rv32gc_ilp32d.tar.xz"
  riscv_gcc.dl_url_vext: "https://syncandshare.lrz.de/dl/fiWBtDLWz17RBc1Yd4VDW7/GCC/default/2024.09.03_gcc14/Ubuntu/{{ ubuntu_version if ubuntu_version is defined else '20.04'}}/rv32gcv_ilp32d.tar.xz"
flags: {}

Do not forget to set your `MLONMCU_HOME` environment variable first if not using the default location!

## Usage

The following examples demonstrates the powerful combination Postprocesses and Features provided by MLonMCU for the described use-case.

### A) Command Line Interface

Add the following arguments to your command:

`--feature log_instrs --config log_instrs.to_file=1 --postprocess analyse_instructions`

**Example:**

In [4]:
!mlonmcu flow run tinymlperf --backend tvmaotplus --target etiss --parallel --config mlif.num_threads=4 -c etiss.print_outputs=0 -c mlif.fuse_ld=lld \
        --config mlif.toolchain=gcc --config run.export_optional=1 --config mlif.strip_strings=1 \
        --post config2cols --config config2cols.limit="['mlif.optimize','mlif.lto']" \
        --post rename_cols --config rename_cols.mapping="{'config_mlif.optimize':'Optimize','config_mlif.lto':'LTO'}" \
        --post filter_cols --config filter_cols.keep="['Model','Simulated Instructions','ROM code','Optimize','LTO']" \
        --config-gen mlif.optimize=0 \
        --config-gen mlif.optimize=1 \
        --config-gen mlif.optimize=2 mlif.lto=0 --config-gen mlif.optimize=2 mlif.lto=1 \
        --config-gen mlif.optimize=3 mlif.lto=0 --config-gen mlif.optimize=3 mlif.lto=1 \
        --config-gen mlif.optimize=s mlif.lto=0 --config-gen mlif.optimize=s mlif.lto=1 \
        --config-gen mlif.optimize=z mlif.lto=0 --config-gen mlif.optimize=z mlif.lto=1 \
        --config-gen mlif.optimize=fast mlif.lto=0 --config-gen mlif.optimize=fast mlif.lto=1

INFO - Loading environment cache from file
INFO - Successfully initialized cache


Traceback (most recent call last):
  File "/tmp/CompareRISCVToolchains-Hboz/venv/bin/mlonmcu", line 33, in <module>
    sys.exit(load_entry_point('mlonmcu', 'console_scripts', 'mlonmcu')())
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/cli/main.py", line 108, in main
    args.func(args)
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/cli/flow.py", line 64, in handle
    args.flow_func(args)
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/cli/run.py", line 55, in handle
    handle_compile(args, context)
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/cli/compile.py", line 83, in handle
    _handle(args, ctx)
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/cli/compile.py", line 46, in _handle
    handle_build(args, ctx=context)
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/cli/build.py", line 115, in handle
    _handle(args, ctx, require_target=require_target)
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/cli/build.py", line 106, in _handle
    new_run.add_target_by_n

The interesting data is exported as an artifact and not shown in the report above. Therefore let's look at the generated files instead.

In [5]:
!mlonmcu export /tmp/exported/ -f --run
!ls /tmp/exported/

INFO - Loading environment cache from file
INFO - Successfully initialized cache
Lookup for session id -1 failed. Available: 


ls: cannot access '/tmp/exported/': No such file or directory


CSV File of the used instruction classes (RISC-V only):

In [6]:
!cat /tmp/exported/analyse_instructions_majors.csv | column -t -s,

cat: /tmp/exported/analyse_instructions_majors.csv: No such file or directory


Frequently used instructions:

In [7]:
!cat /tmp/exported/analyse_instructions_seq1.csv | column -t -s,

cat: /tmp/exported/analyse_instructions_seq1.csv: No such file or directory


### B) Python Scripting

Python Imports

In [8]:
from tempfile import TemporaryDirectory
from pathlib import Path
import pandas as pd

from mlonmcu.context.context import MlonMcuContext
from mlonmcu.session.run import RunStage

Benchmark Configuration

In [9]:
FRONTEND = "tflite"
MODEL = "sine_model"
BACKEND = "tvmaotplus"
PLATFORM = "mlif"
TARGET = "etiss"
FEATURES = ["log_instrs"]
CONFIG = {"log_instrs.to_file": True}
POSTPROCESSES = ["analyse_instructions"]

Initialize and run a single benchmark

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [10]:
with MlonMcuContext() as context:
    with context.create_session() as session:
        run = session.create_run(config=CONFIG)
        run.add_features_by_name(FEATURES, context=context)
        run.add_frontend_by_name(FRONTEND, context=context)
        run.add_model_by_name(MODEL, context=context)
        run.add_backend_by_name(BACKEND, context=context)
        run.add_platform_by_name(PLATFORM, context=context)
        run.add_target_by_name(TARGET, context=context)
        run.add_postprocesses_by_name(POSTPROCESSES)
        session.process_runs(context=context)
        report = session.get_reports()
assert not session.failing
report.df

INFO - Loading environment cache from file


INFO - Successfully initialized cache


RuntimeError: Dependency cache miss for required key 'etiss.src_dir'. Try re-running `mlonmcu setup`.

The report is available as pandas dataframe.

Let's extract the artifacts created by the `analyse_instructions` postprocess:

In [ ]:
artifacts = session.runs[0].artifacts_per_stage[RunStage.POSTPROCESS]["default"]

**Visualization of used Instruction types**

In [ ]:
with TemporaryDirectory() as tmpdirname:
    dest = Path(tmpdirname) / "analyse_instructions_majors.csv"
    artifacts[0].export(dest)
    majors_df = pd.read_csv(dest)
    majors_df = majors_df.drop("Probablity", axis=1)
    majors_df.plot.bar(x="Major", title=f"Top Major Opcodes")

**Most used (groups of) intructions in the program**

In [ ]:
with TemporaryDirectory() as tmpdirname:
    for i in range(1, 4):
        dest = Path(tmpdirname) / f"analyse_instructions_seq{i}.csv"
        artifacts[i].export(dest)
        seq_df = pd.read_csv(dest)
        top = len(seq_df)
        seq_df = seq_df.drop("Probablity", axis=1)
        seq_df.plot.bar(x="Sequence", title=f"Top {top} {i}-length sequences")